# 闭包和装饰器

Python 中的装饰器大致分为**函数装饰器**和**类装饰器**。顾名思义，函数装饰器是装饰函数的，类装饰器是装饰类的，不论是函数装饰器还是类装饰器，它们的实现都是函数。

装饰器是可调用的对象，其参数是另一个函数（被装饰的函数）或另一个类（被装饰的类）。装饰器可能会处理被装饰的函数或类，然后把它返回，或者将其替换成另一个函数或类或可调用对象。

装饰器有两大特性：

- 能把被装饰的函数或类替换成其他函数或类
- 它们在被装饰的函数或类定义之后立即运行。这通常是在*导入时*，即 Python 加载模块时

## 变量作用域

在 Python 中，使用 `class`、`def`、`lambda` 会引入一个**本地作用域**（Local），而且本地作用域是可以嵌套的。内层的变量会屏蔽外层，内层找不到的变量会去外层找。

**全局作用域**（Global）就是模块内的作用域，它的作用范围是单一文件内。

内置函数的作用域就是 `int()`、`list()` 之类的内置函数。

Python 的变量作用域遵循 **LEGB** 法则。即当 Python 遇到一个变量的话它会按照这样的顺序进行搜索：

1. 本地作用域（Local）
2. 当前作用域被嵌入的本地作用域（Enclosing locals）
3. 全局/模块作用域（Global）
4. 内置作用域（Built-in）

如下：

In [5]:
a1 = 1  # Global
class Foo:
    a2 = 1  # Local
    def func():
        a3 = 1  # Local
        def _func():
            a5 = 1  # Local
        for i in range(3):
            a4 = 1  # Local，和 a3 在同一个作用域，因为 for 不会引入新的作用域
        print(i)  # i=2

上述代码中作用域的范围是 `a1` > `a2` > `a3` == `a4` > `a5`，除此之外，因为 `for` 不会引入新的作用域，所以在 `for` 循环外依然可以访问 `i`。

再看下面一个例子，首先它会找到局部变量 `a`，并打印 `a` 的值 `3`，其次它会尝试找局部变量 `b`，发现 `b` 在被赋值之前就被引用了，所以会抛出一个错误。

In [6]:
b = 6
def f1(a):
    print(a)
    print(b)
    b = 9
f1(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

如果在函数中赋值时想让解释器把 `b` 当成全局变量，要使用 `global` 声明，即在引用 `b` 之前加上 `global b`。

如果去掉 `b = 9` 那条赋值语句，解释器会先尝试找局部变量 `b`，发现找不到，然后按照 LEGB 法则，依次找到全局变量 `b`，所以会正常打印 `b` 的值 `6`，不会出现错误。

## 闭包

闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

**自由变量**（free variable）指未在本地作用域中绑定的变量。

注意，只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量。

In [14]:
def make_averager():
    
    # averager 的闭包延伸到那个函数的作用域之外，包含自由变量 series 的绑定
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()

print(avg(10))
print(avg(11))

10.0
10.5


通俗点说，闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的**非全局变量**。注意这里的非全局变量，函数引用定义体之外的全局变量很常见，但要引用定义体之外的非全局变量往往就和**函数的嵌套**相关联。

函数引用定义体之外的非全局变量，除了在函数的嵌套中出现外，还会出现在类中，从这个角度看，闭包是类的方便实现。

## 函数装饰器

函数装饰器用于在源码中“标记”函数，以某种方式增强函数的行为。

函数装饰器在导入模块时立即执行，而被装饰的函数只在明确调用时运行。这突出了 Python 程序员所说的*导入时*和*运行时*之间的区别。

装饰器通常在一个模块中定义，然后应用到其他模块中的函数上。

大多数装饰器会在内部定义一个函数，然后将其返回，替换被装饰的函数。通常来说，这两者接受相同的参数，而且返回被装饰的函数本该返回的值，同时还会做些额外操作。

函数装饰器包括最简单的**注册装饰器**和较复杂的**参数化装饰器**。

### 注册装饰器

In [31]:
def outer(func):
    
    def inner(*args, **kwargs):
        
        print('before')
        
        result = func(*args, **kwargs)
        
        print('after')
        
        return result
    
    return inner

@outer  # func = outer(func)
def func1():
    print('func')

@outer
def func2(a):
    print(a)

func1()
print('------')
print(func1.__name__)
print('======')
func2('hello')
print('------')
print(func2.__name__)

before
func
after
------
inner
before
hello
after
------
inner


### 参数化装饰器

## 类装饰器

参见第 21 章